In [1]:
import os
os.chdir('./drive/My Drive/AFC_Assignment_2/')
!ls

actor_10_emotional_contours.pickle  actor_23_labeled_contours.pickle
actor_10_labeled_contours.pickle    actor_23_neutral_contours.pickle
actor_10_neutral_contours.pickle    actor_24_emotional_contours.pickle
actor_11_emotional_contours.pickle  actor_24_labeled_contours.pickle
actor_11_labeled_contours.pickle    actor_24_neutral_contours.pickle
actor_11_neutral_contours.pickle    actor_2_emotional_contours.pickle
actor_12_emotional_contours.pickle  actor_2_labeled_contours.pickle
actor_12_labeled_contours.pickle    actor_2_neutral_contours.pickle
actor_12_neutral_contours.pickle    actor_3_emotional_contours.pickle
actor_13_emotional_contours.pickle  actor_3_labeled_contours.pickle
actor_13_labeled_contours.pickle    actor_3_neutral_contours.pickle
actor_13_neutral_contours.pickle    actor_4_emotional_contours.pickle
actor_14_emotional_contours.pickle  actor_4_labeled_contours.pickle
actor_14_labeled_contours.pickle    actor_4_neutral_contours.pickle
actor_14_neutral_contours.pickle   

In [2]:
!pwd

/content/drive/My Drive/AFC_Assignment_2


In [0]:
!unzip Audio_Speech_Actors_01-24.zip

Archive:  Audio_Speech_Actors_01-24.zip
   creating: Actor_01/
  inflating: Actor_01/03-01-01-01-01-01-01.wav  
  inflating: Actor_01/03-01-01-01-01-02-01.wav  
  inflating: Actor_01/03-01-01-01-02-01-01.wav  
  inflating: Actor_01/03-01-01-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-01-01-01-01.wav  
  inflating: Actor_01/03-01-02-01-01-02-01.wav  
  inflating: Actor_01/03-01-02-01-02-01-01.wav  
  inflating: Actor_01/03-01-02-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-02-01-01-01.wav  
  inflating: Actor_01/03-01-02-02-01-02-01.wav  
  inflating: Actor_01/03-01-02-02-02-01-01.wav  
  inflating: Actor_01/03-01-02-02-02-02-01.wav  
  inflating: Actor_01/03-01-03-01-01-01-01.wav  
  inflating: Actor_01/03-01-03-01-01-02-01.wav  
  inflating: Actor_01/03-01-03-01-02-01-01.wav  
  inflating: Actor_01/03-01-03-01-02-02-01.wav  
  inflating: Actor_01/03-01-03-02-01-01-01.wav  
  inflating: Actor_01/03-01-03-02-01-02-01.wav  
  inflating: Actor_01/03-01-03-02-02-01-01.wav  
  infl

In [4]:
!ls -l ./data/Actor_01 | wc -l

61


In [3]:
!pip install soundfile

In [0]:
from numpy.fft import rfft
import numpy as np
from scipy.signal import blackmanharris, correlate
# import soundfile
import sys
import math
import pickle
import wave
import pylab
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import mixture
from sklearn.metrics import accuracy_score

# **Utility functions**

In [0]:
def parabolic(signal, value):
    # referred from https://github.com/endolith/waveform_analysis/blob/66809614b1fc985e694af1720341035316a5ac8e/waveform_analysis/_common.py#L146
    if(value-1<0):
      value+=1
    if(value+1>=len(signal)):
      value-=(value+1)-(len(signal)-1)
    x_ = (1/2)*(signal[value-1]-signal[value+1])/(signal[value-1]-2*signal[value]+signal[value+1])+value
    y_ = signal[value]-(1/4)*(signal[value-1]-signal[value+1])*(x_-value)
    return x_, y_

def freq_from_autocorr(signal, sampling_rate):
  # referred from https://gist.github.com/endolith/255291
    differenced_signal = np.diff(correlate(sig, sig, mode='full')[len(correlate(sig, sig, mode='full'))//2 : ])

    try:
      start = np.nonzero(differenced_signal>0)[0][0]
    except IndexError as e:
      start = 0

    local_maxima = np.argmax(correlate(sig, sig, mode='full'[start:]))+start
    vertex_x, vertex_y = parabolic(correlate(sig, sig, mode='full'), local_maxima)
  
    if(math.isnan(vertex_x)):
      vertex_x = 0
    return sampling_rate/(vertex_x+sys.float_info.epsilon)

In [0]:
def split_signal_into_frames(signal, window_size, overlap):
  windows = []

  for win in range(0, len(signal), window_size-overlap):
    start_ind = win
    end_ind = start_ind+window_size

    if(end_ind<len(signal)):
      window = signal[start_ind:end_ind]
    else:
      window_ = signal[start_ind:len(signal)]
      window = list(window_) + [0]*(window_size-len(window_))
    windows.append(list(window))
  return windows

def get_f0_contours(signal, sampling_rate, window_size, overlap):
    frames = split_signal_into_frames(signal, window_size, overlap)
    f0_contours = []

    for frame in frames:
      f0_contours.append(freq_from_autocorr(frame, sampling_rate))

    return f0_contours, f0_contours[:-3]

In [8]:
# signal, sampling_rate = soundfile.read('./data/Actor_01/03-01-01-01-01-01-01.wav')
wav = wave.open('./data/Actor_01/03-01-01-01-01-01-01.wav', 'r')
frames = wav.readframes(-1)
signal = pylab.fromstring(frames, 'Int16')
sampling_rate = wav.getframerate()
_, f0_contours = get_f0_contours(signal, sampling_rate, 1920, 1440)

print(f0_contours)
q75, q25 = np.percentile(f0_contours, [75 ,25])

print(q75)
print(q25)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: overflow encountered in short_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in short_scalars


[4800.0, 2.161727821137838e+20, 2.161727821137838e+20, 12000.0, 12000.0, 3164.835164835165, 3178.8079470198677, 8000.0, 8000.0, 8000.0, 12000.0, 12000.0, 1777.7777777777778, 5333.333333333333, 5433.962264150943, 5433.962264150943, 5433.962264150943, 5517.241379310345, 705.8823529411765, 1608.9385474860335, 1608.9385474860335, 1608.9385474860335, 4033.613445378151, 6000.0, 10666.666666666666, 4298.507462686567, 12000.0, 4298.507462686567, 2198.4732824427483, 2198.4732824427483, 2198.4732824427483, 4848.484848484848, 205.12820512820514, 60.2258469259724, 1846.1538461538462, 1846.1538461538462, 1846.1538461538462, 1846.1538461538462, 6857.142857142857, 6857.142857142857, 6857.142857142857, 888.8888888888889, 888.8888888888889, 3200.0, 3200.0, 3200.0, 3200.0, 2086.9565217391305, 2086.9565217391305, 2.161727821137838e+20, 77.41935483870968, 77.41935483870968, 64.6900269541779, 64.6900269541779, 64.6900269541779, 2412.060301507538, 2412.060301507538, 2412.060301507538, 2412.060301507538, 300

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


# **F0 contours extraction for the entire dataset**

In [50]:
data_dir = './data/'
actor_dir = 'Actor_'

num_actors = 24

actor_contour_dict = {}
actor_neutral_dict = {}
actor_emotional_dict = {}

for i in range(1, num_actors+1):
    print('Actor '+str(i))
    if(i<10):
      folder = data_dir+actor_dir+'0'+str(i)+'/'
    else:
      folder = data_dir+actor_dir+str(i)+'/'

    files =  os.listdir(folder)

    actor_list = []
    neutral_list = []
    emotional_list = []

    for idx, file in enumerate(files):
        print(file)
        emotion = int(file[6:8])
        if(emotion==1):
            emotion=0
        else:
            emotion=1
        # signal, sampling_rate = soundfile.read(folder+file)
        wav = wave.open(folder+file, 'r')
        frames = wav.readframes(-1)
        signal = pylab.fromstring(frames, 'Int16')
        sampling_rate = wav.getframerate()
        _, f0_contours = get_f0_contours(signal, sampling_rate, int((sampling_rate/1000)*40), int((sampling_rate/1000)*30))
        print(np.asarray(signal).shape, sampling_rate, len(f0_contours))

        actor_list.append([f0_contours, emotion])

        if(emotion==0):
            neutral_list.append(f0_contours)
        else:
            emotional_list.append(f0_contours)

    actor_contour_dict[i] = actor_list
    actor_neutral_dict[i] = neutral_list
    actor_emotional_dict[i] = emotional_list

    pickle.dump(actor_list, open('actor_'+str(i)+'_labeled_contours.pickle', 'wb'))
    pickle.dump(neutral_list, open('actor_'+str(i)+'_neutral_contours.pickle', 'wb'))
    pickle.dump(emotional_list, open('actor_'+str(i)+'_emotional_contours.pickle', 'wb'))

pickle.dump(actor_contour_dict, open('actor_contour_dict.pickle', 'wb'))
pickle.dump(actor_neutral_dict, open('actor_neutral_dict.pickle', 'wb'))
pickle.dump(actor_emotional_dict, open('actor_emotional_dict.pickle', 'wb'))

Actor 1
03-01-02-01-02-02-01.wav


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: overflow encountered in short_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: RuntimeWarning: overflow encountered in short_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


(166567,) 48000 345
03-01-01-01-02-01-01.wav
(156956,) 48000 324
03-01-01-01-01-02-01.wav


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in double_scalars


(160160,) 48000 331
03-01-02-01-01-02-01.wav
(345944,) 48000 718
03-01-01-01-02-02-01.wav
(152152,) 48000 314
03-01-02-01-01-01-01.wav
(169769,) 48000 351
03-01-02-01-02-01-01.wav
(168168,) 48000 348
03-01-01-01-01-01-01.wav
(158558,) 48000 328
03-01-02-02-01-01-01.wav
(177778,) 48000 368
03-01-04-02-02-01-01.wav
(179380,) 48000 371
03-01-04-01-01-02-01.wav
(171371,) 48000 355
03-01-03-02-01-02-01.wav
(174574,) 48000 361
03-01-03-01-02-02-01.wav
(177778,) 48000 368
03-01-04-01-01-01-01.wav
(184184,) 48000 381
03-01-03-02-02-02-01.wav
(188989,) 48000 391
03-01-03-01-01-02-01.wav
(166566,) 48000 345
03-01-03-01-01-01-01.wav
(166567,) 48000 345
03-01-03-01-02-01-01.wav
(169769,) 48000 351
03-01-02-02-02-01-01.wav
(201802,) 48000 418
03-01-04-02-01-02-01.wav
(177778,) 48000 368
03-01-04-01-02-01-01.wav
(168168,) 48000 348
03-01-03-02-02-01-01.wav
(188988,) 48000 391
03-01-04-02-01-01-01.wav
(182582,) 48000 378
03-01-03-02-01-01-01.wav
(176176,) 48000 365
03-01-04-02-02-02-01.wav
(179379,) 

# **Subsamping reference neutral database from existing dataset**

In [0]:
reference_neutral_features = []
training_neutral_features = []
num_actors = 24

file_template = ['actor_', '_neutral_contours.pickle']

for i in range(1, num_actors+1):
    num = str(i)
    feats = pickle.load(open(file_template[0]+num+file_template[1], 'rb'))
    reference_neutral_features.append(feats[0])
    training_neutral_features.append(feats[1:])

pickle.dump(reference_neutral_features, open('reference_neutral_features.pickle', 'wb'))
pickle.dump(training_neutral_features, open('training_neutral_features.pickle', 'wb'))

# **Computing F0_ref value for IFN updates and normalization parameters**

In [0]:
ref_neutral_features = pickle.load(open('reference_neutral_features.pickle', 'rb'))

f0_averages = []

for feat in ref_neutral_features:
    f0_averages.append(np.mean(np.clip(feat, 0, 60000)))

pickle.dump([f0_averages, np.mean(f0_averages)], open('f0_ref.pickle', 'wb'))

# **Computing 7 F0 statistic features**

In [45]:
actor_contour_dict = pickle.load(open('actor_contour_dict.pickle', 'rb'))
actor_feature_dict = {}
training_emotional_features = []

for key in actor_contour_dict.keys():
    print('Actor', str(key))
    feats_and_labels_array = actor_contour_dict[key]
    actor_list = []
    emotional_feats = []
    for i in range(len(feats_and_labels_array)):
          f0_contours = feats_and_labels_array[i][0]
          label = feats_and_labels_array[i][1]
          
          f0_contours = np.clip(f0_contours, 0, 60000)

          # features are in the order: [mean, std, range, q1, q3, iqr, median]

          f0_mean = np.mean(f0_contours)
          f0_std = np.std(f0_contours)
          f0_range = max(f0_contours)-min(f0_contours)
          f0_q3, f0_q1 = np.percentile(f0_contours, [75, 25])
          f0_iqr = f0_q3-f0_q1
          f0_median = np.median(f0_contours)

          if(label==1):
              emotional_feats.append([f0_mean, f0_std, f0_range, f0_q1, f0_q3, f0_iqr, f0_median])

          actor_list.append([[f0_mean, f0_std, f0_range, f0_q1, f0_q3, f0_iqr, f0_median], label])

    actor_feature_dict[key] = actor_list
    training_emotional_features.append(emotional_feats)

pickle.dump(actor_feature_dict, open('actor_feature_dict.pickle', 'wb'))
pickle.dump(training_emotional_features, open('train_emotional_features.pickle', 'wb'))

Actor 1
Actor 2
Actor 3
Actor 4
Actor 5
Actor 6
Actor 7
Actor 8
Actor 9
Actor 10
Actor 11
Actor 12
Actor 13
Actor 14
Actor 15
Actor 16
Actor 17
Actor 18
Actor 19
Actor 20
Actor 21
Actor 22
Actor 23
Actor 24


In [0]:
training_neutral_features = pickle.load(open('training_neutral_features.pickle', 'rb'))
reference_neutral_features = pickle.load(open('reference_neutral_features.pickle', 'rb'))

train_neutral_features = []
ref_neutral_features = []

for i in range(len(reference_neutral_features)):
    f0_contours = np.clip(reference_neutral_features[i], 0, 60000)

    # features are in the order: [mean, std, range, q1, q3, iqr, median]
    f0_mean = np.mean(f0_contours)
    f0_std = np.std(f0_contours)
    f0_range = max(f0_contours)-min(f0_contours)
    f0_q3, f0_q1 = np.percentile(f0_contours, [75, 25])
    f0_iqr = f0_q3-f0_q1
    f0_median = np.median(f0_contours)

    ref_neutral_features.append([f0_mean, f0_std, f0_range, f0_q1, f0_q3, f0_iqr, f0_median])

for i in range(len(training_neutral_features)):
    curr_set = []

    for j in range(3):
        f0_contours = np.clip(training_neutral_features[i][j], 0, 60000)

        # features are in the order: [mean, std, range, q1, q3, iqr, median]
        f0_mean = np.mean(f0_contours)
        f0_std = np.std(f0_contours)
        f0_range = max(f0_contours)-min(f0_contours)
        f0_q3, f0_q1 = np.percentile(f0_contours, [75, 25])
        f0_iqr = f0_q3-f0_q1
        f0_median = np.median(f0_contours)

        curr_set.append([f0_mean, f0_std, f0_range, f0_q1, f0_q3, f0_iqr, f0_median])

    train_neutral_features.append(curr_set)

pickle.dump(ref_neutral_features, open('ref_neutral_features.pickle', 'wb'))
pickle.dump(train_neutral_features, open('train_neutral_features.pickle', 'wb'))

# **Training with Optimal Normalization**

In [0]:
# split into training and held out sets
actor_feature_dict = pickle.load(open('actor_feature_dict.pickle', 'rb'))
f0_ref = pickle.load(open('f0_ref.pickle', 'rb'))[1]
training_neutral_features = pickle.load(open('train_neutral_features.pickle',  'rb'))
training_emotional_features = pickle.load(open('train_emotional_features.pickle', 'rb'))
reference_neutral_features = pickle.load(open('ref_neutral_features.pickle', 'rb'))

training_neutral_subset, test_neutral_subset = training_neutral_features[:18], training_neutral_features[18:]
training_emotional_subset, test_emotional_subset = training_emotional_features[:18], training_emotional_features[18:]

In [0]:
# sampling emotional dataset
num_train_actors = 18
def sample_emotional_features(training_emotional_subset):
    return_subset = []

    for actor_id in range(num_train_actors):
        subset = np.random.choice(np.arange(len(training_emotional_subset[actor_id])), 3, replace=False)
        return_subset.append([training_emotional_subset[actor_id][subset[0]], training_emotional_subset[actor_id][subset[1]], training_emotional_subset[actor_id][subset[2]]])

    return return_subset

In [0]:
# format test data in usable format
def create_test_data(test_neutral_subset, test_emotional_subset):
    test_neutral_subset = np.asarray(test_neutral_subset)
    test_emotional_subset = np.asarray(test_emotional_subset)
    
    test_x = np.zeros((test_neutral_subset.shape[0]*test_neutral_subset.shape[1] + test_emotional_subset.shape[0]*test_emotional_subset.shape[1], 7))
    test_y = np.zeros(test_neutral_subset.shape[0]*test_neutral_subset.shape[1] + test_emotional_subset.shape[0]*test_emotional_subset.shape[1])
    test_idx = 0    

    for i in range(len(test_neutral_subset)):
          for j in range(test_neutral_subset.shape[1]):
              test_x[test_idx] = test_neutral_subset[i][j]
              test_y[test_idx] = 0
              test_idx+=1

    for i in range(len(test_emotional_subset)):
          for j in range(test_emotional_subset.shape[1]):
              test_x[test_idx] = test_emotional_subset[i][j]
              test_y[test_idx] = 1
              test_idx+=1       
    
    return test_x, test_y

In [0]:
# training GMM on reference neutral dataset
trained_GMMs = []

for i in range(7):
    gmm = mixture.GaussianMixture(n_components=2)
    gmm.fit(np.asarray(reference_neutral_features)[:, i].reshape(-1, 1))
    trained_GMMs.append(gmm)

In [0]:
# compute optimal normalization parameters
sampled_emotional_subset = np.asarray(sample_emotional_features(training_emotional_subset))
sampled_neutral_subset = np.asarray(training_neutral_subset)

per_speaker_normalizers = []

for actor_idx in range(len(sampled_neutral_subset)):
    norm_val = np.mean([np.mean(sampled_neutral_subset[actor_idx][0]), np.mean(sampled_neutral_subset[actor_idx][1]), np.mean(sampled_neutral_subset[actor_idx][2])])
    per_speaker_normalizers.append(norm_val)    

In [0]:
# features as fitness measures of gmm likelihoods
for actor_idx in range(len(sampled_neutral_subset)):
    sampled_neutral_subset[actor_idx] *= per_speaker_normalizers[actor_idx]
    # sampled_emotional_subset[actor_idx] *= per_speaker_normalizers[actor_idx]

fitness_measures_emotional = np.zeros_like(sampled_emotional_subset)
fitness_measures_neutral = np.zeros_like(sampled_neutral_subset)

for i in range(num_train_actors):
    curr_emotional_features = sampled_emotional_subset[i]
    curr_neutral_features = sampled_neutral_subset[i]

    for j in range(7):
        fitness_measures_emotional[i, :, j] = np.exp(trained_GMMs[j].score_samples(sampled_emotional_subset[i][:, j].reshape(-1, 1)))
        fitness_measures_neutral[i, :, j] = np.exp(trained_GMMs[j].score_samples(sampled_neutral_subset[i][:, j].reshape(-1, 1)))

In [0]:
# create feature arrays for LDA classifier

train_feature_matrix = np.zeros((72, 7))
val_feature_matrix = np.zeros((36, 7))
train_labels = np.zeros(72)
val_labels = np.zeros(36)

train_feat_ind = 0
val_feat_ind = 0

for i in range(num_train_actors):
    train_feature_matrix[train_feat_ind] = fitness_measures_neutral[i][0]
    train_labels[train_feat_ind] = 0
    train_feat_ind+=1
    train_feature_matrix[train_feat_ind] = fitness_measures_neutral[i][1]
    train_labels[train_feat_ind] = 0
    train_feat_ind+=1
    val_feature_matrix[val_feat_ind] = fitness_measures_neutral[i][2]
    val_labels[val_feat_ind] = 0
    val_feat_ind+=1

for i in range(num_train_actors):
    train_feature_matrix[train_feat_ind] = fitness_measures_emotional[i][0]
    train_labels[train_feat_ind] = 1
    train_feat_ind+=1
    train_feature_matrix[train_feat_ind] = fitness_measures_emotional[i][1]
    train_labels[train_feat_ind] = 1
    train_feat_ind+=1
    val_feature_matrix[val_feat_ind] = fitness_measures_emotional[i][2]
    val_labels[val_feat_ind] = 1
    val_feat_ind+=1

In [29]:
# fitting ldc on fitness measure features
ldc = LinearDiscriminantAnalysis()
ldc.fit(train_feature_matrix, train_labels)

test_x, test_y = create_test_data(test_neutral_subset, test_emotional_subset)

print('Validation accuracy: '+str(accuracy_score(val_labels, ldc.predict(val_feature_matrix))))
print('Test accuracy: '+str(accuracy_score(train_labels, ldc.predict(train_feature_matrix))))
print('Test accuracy: '+str(accuracy_score(test_y, ldc.predict(test_x))))

Validation accuracy: 1.0
Test accuracy: 1.0
Test accuracy: 0.9491525423728814


# **Training with No Normalization**

In [0]:
# split into training and held out sets
actor_feature_dict = pickle.load(open('actor_feature_dict.pickle', 'rb'))
f0_ref = pickle.load(open('f0_ref.pickle', 'rb'))
training_neutral_features = pickle.load(open('train_neutral_features.pickle',  'rb'))
training_emotional_features = pickle.load(open('train_emotional_features.pickle', 'rb'))
reference_neutral_features = pickle.load(open('ref_neutral_features.pickle', 'rb'))

training_neutral_subset, test_neutral_subset = training_neutral_features[:18], training_neutral_features[18:]
training_emotional_subset, test_emotional_subset = training_emotional_features[:18], training_emotional_features[18:]

In [0]:
# sampling emotional dataset
num_train_actors = 18
def sample_emotional_features(training_emotional_subset):
    return_subset = []

    for actor_id in range(num_train_actors):
        subset = np.random.choice(np.arange(len(training_emotional_subset[actor_id])), 3, replace=False)
        return_subset.append([training_emotional_subset[actor_id][subset[0]], training_emotional_subset[actor_id][subset[1]], training_emotional_subset[actor_id][subset[2]]])

    return return_subset

In [0]:
# format test data in usable format
def create_test_data(test_neutral_subset, test_emotional_subset):
    test_neutral_subset = np.asarray(test_neutral_subset)
    test_emotional_subset = np.asarray(test_emotional_subset)
    
    test_x = np.zeros((test_neutral_subset.shape[0]*test_neutral_subset.shape[1] + test_emotional_subset.shape[0]*test_emotional_subset.shape[1], 7))
    test_y = np.zeros(test_neutral_subset.shape[0]*test_neutral_subset.shape[1] + test_emotional_subset.shape[0]*test_emotional_subset.shape[1])
    test_idx = 0    

    for i in range(len(test_neutral_subset)):
          for j in range(test_neutral_subset.shape[1]):
              test_x[test_idx] = test_neutral_subset[i][j]
              test_y[test_idx] = 0
              test_idx+=1

    for i in range(len(test_emotional_subset)):
          for j in range(test_emotional_subset.shape[1]):
              test_x[test_idx] = test_emotional_subset[i][j]
              test_y[test_idx] = 1
              test_idx+=1       
    
    return test_x, test_y

In [0]:
# training GMM on reference neutral dataset
trained_GMMs = []

for i in range(7):
    gmm = mixture.GaussianMixture(n_components=2)
    gmm.fit(np.asarray(reference_neutral_features)[:, i].reshape(-1, 1))
    trained_GMMs.append(gmm)

In [0]:
# features as fitness measures of gmm likelihoods

sampled_emotional_subset = np.asarray(sample_emotional_features(training_emotional_subset))
sampled_neutral_subset = np.asarray(training_neutral_subset)

fitness_measures_emotional = np.zeros_like(sampled_emotional_subset)
fitness_measures_neutral = np.zeros_like(sampled_neutral_subset)

for i in range(num_train_actors):
    curr_emotional_features = sampled_emotional_subset[i]
    curr_neutral_features = sampled_neutral_subset[i]

    for j in range(7):
        fitness_measures_emotional[i, :, j] = np.exp(trained_GMMs[j].score_samples(sampled_emotional_subset[i][:, j].reshape(-1, 1)))
        fitness_measures_neutral[i, :, j] = np.exp(trained_GMMs[j].score_samples(sampled_neutral_subset[i][:, j].reshape(-1, 1)))

In [0]:
# create feature arrays for LDA classifier

train_feature_matrix = np.zeros((72, 7))
val_feature_matrix = np.zeros((36, 7))
train_labels = np.zeros(72)
val_labels = np.zeros(36)

train_feat_ind = 0
val_feat_ind = 0

for i in range(num_train_actors):
    train_feature_matrix[train_feat_ind] = fitness_measures_neutral[i][0]
    train_labels[train_feat_ind] = 0
    train_feat_ind+=1
    train_feature_matrix[train_feat_ind] = fitness_measures_neutral[i][1]
    train_labels[train_feat_ind] = 0
    train_feat_ind+=1
    val_feature_matrix[val_feat_ind] = fitness_measures_neutral[i][2]
    val_labels[val_feat_ind] = 0
    val_feat_ind+=1

for i in range(num_train_actors):
    train_feature_matrix[train_feat_ind] = fitness_measures_emotional[i][0]
    train_labels[train_feat_ind] = 1
    train_feat_ind+=1
    train_feature_matrix[train_feat_ind] = fitness_measures_emotional[i][1]
    train_labels[train_feat_ind] = 1
    train_feat_ind+=1
    val_feature_matrix[val_feat_ind] = fitness_measures_emotional[i][2]
    val_labels[val_feat_ind] = 1
    val_feat_ind+=1

In [44]:
# fitting ldc on fitness measure features
ldc = LinearDiscriminantAnalysis()
ldc.fit(train_feature_matrix, train_labels)

test_x, test_y = create_test_data(test_neutral_subset, test_emotional_subset)

print('Validation accuracy: '+str(accuracy_score(val_labels, ldc.predict(val_feature_matrix))))
print('Train accuracy: '+str(accuracy_score(train_labels, ldc.predict(train_feature_matrix))))
print('Test accuracy: '+str(accuracy_score(test_y, ldc.predict(test_x))))

Validation accuracy: 0.6111111111111112
Train accuracy: 0.6805555555555556
Test accuracy: 0.268361581920904


# **Training with IFN**

In [0]:
# split into training and held out sets
actor_feature_dict = pickle.load(open('actor_feature_dict.pickle', 'rb'))
f0_ref = pickle.load(open('f0_ref.pickle', 'rb'))[1]
training_neutral_features = pickle.load(open('train_neutral_features.pickle',  'rb'))
training_emotional_features = pickle.load(open('train_emotional_features.pickle', 'rb'))
reference_neutral_features = pickle.load(open('ref_neutral_features.pickle', 'rb'))

training_neutral_subset, test_neutral_subset = training_neutral_features[:18], training_neutral_features[18:]
training_emotional_subset, test_emotional_subset = training_emotional_features[:18], training_emotional_features[18:]

In [0]:
# sampling emotional dataset
num_train_actors = 18
def sample_emotional_features(training_emotional_subset):
    return_subset = []

    for actor_id in range(num_train_actors):
        subset = np.random.choice(np.arange(len(training_emotional_subset[actor_id])), 3, replace=False)
        return_subset.append([training_emotional_subset[actor_id][subset[0]], training_emotional_subset[actor_id][subset[1]], training_emotional_subset[actor_id][subset[2]]])

    return return_subset

In [0]:
# format test data in usable format
def create_test_data(test_neutral_subset, test_emotional_subset):
    test_neutral_subset = np.asarray(test_neutral_subset)
    test_emotional_subset = np.asarray(test_emotional_subset)
    
    test_x = np.zeros((test_neutral_subset.shape[0]*test_neutral_subset.shape[1] + test_emotional_subset.shape[0]*test_emotional_subset.shape[1], 7))
    test_y = np.zeros(test_neutral_subset.shape[0]*test_neutral_subset.shape[1] + test_emotional_subset.shape[0]*test_emotional_subset.shape[1])
    test_idx = 0    

    for i in range(len(test_neutral_subset)):
          for j in range(test_neutral_subset.shape[1]):
              test_x[test_idx] = test_neutral_subset[i][j]
              test_y[test_idx] = 0
              test_idx+=1

    for i in range(len(test_emotional_subset)):
          for j in range(test_emotional_subset.shape[1]):
              test_x[test_idx] = test_emotional_subset[i][j]
              test_y[test_idx] = 1
              test_idx+=1       
    
    return test_x, test_y

In [0]:
# training GMM on reference neutral dataset
def train_gmss(reference_neutral_features):
    trained_GMMs = []

    for i in range(7):
        gmm = mixture.GaussianMixture(n_components=2)
        gmm.fit(np.asarray(reference_neutral_features)[:, i].reshape(-1, 1))
        trained_GMMs.append(gmm)

    return trained_GMMs

In [0]:
# get per speaker normalization parameter
def get_per_speaker_normalization_parameters(sampled_neutral_subset):
    for actor_idx in range(len(sampled_neutral_subset)):
        sampled_neutral_subset[actor_idx] *= per_speaker_normalizers[actor_idx]

    return sampled_neutral_subset

In [0]:
# sample emotional data to prevent class imbalance
def get_sampled_data(training_emotional_subset, training_neutral_subset):
    sampled_emotional_subset = np.asarray(sample_emotional_features(training_emotional_subset))
    sampled_neutral_subset = np.asarray(training_neutral_subset)

    return sampled_emotional_subset, sampled_neutral_subset

In [0]:
# features as fitness measures of gmm likelihoods
def get_fitness_measures(sampled_emotional_subset, sampled_neutral_subset, trained_gmms):
    fitness_measures_emotional = np.zeros_like(sampled_emotional_subset)
    fitness_measures_neutral = np.zeros_like(sampled_neutral_subset)

    for i in range(num_train_actors):
        curr_emotional_features = sampled_emotional_subset[i]
        curr_neutral_features = sampled_neutral_subset[i]

        for j in range(7):
            fitness_measures_emotional[i, :, j] = np.exp(trained_GMMs[j].score_samples(sampled_emotional_subset[i][:, j].reshape(-1, 1)))
            fitness_measures_neutral[i, :, j] = np.exp(trained_GMMs[j].score_samples(sampled_neutral_subset[i][:, j].reshape(-1, 1)))

    return fitness_measures_emotional, fitness_measures_neutral

In [0]:
# create feature arrays for LDA classifier
def create_train_val_xy(fitness_measures_emotional, fitness_measures_neutral):
    train_feature_matrix = np.zeros((72, 7))
    val_feature_matrix = np.zeros((36, 7))
    train_labels = np.zeros(72)
    val_labels = np.zeros(36)

    train_feat_ind = 0
    val_feat_ind = 0

    for i in range(num_train_actors):
        train_feature_matrix[train_feat_ind] = fitness_measures_neutral[i][0]
        train_labels[train_feat_ind] = 0
        train_feat_ind+=1
        train_feature_matrix[train_feat_ind] = fitness_measures_neutral[i][1]
        train_labels[train_feat_ind] = 0
        train_feat_ind+=1
        val_feature_matrix[val_feat_ind] = fitness_measures_neutral[i][2]
        val_labels[val_feat_ind] = 0
        val_feat_ind+=1

    for i in range(num_train_actors):
        train_feature_matrix[train_feat_ind] = fitness_measures_emotional[i][0]
        train_labels[train_feat_ind] = 1
        train_feat_ind+=1
        train_feature_matrix[train_feat_ind] = fitness_measures_emotional[i][1]
        train_labels[train_feat_ind] = 1
        train_feat_ind+=1
        val_feature_matrix[val_feat_ind] = fitness_measures_emotional[i][2]
        val_labels[val_feat_ind] = 1
        val_feat_ind+=1

    return train_feature_matrix, val_feature_matrix, train_labels, val_labels

In [0]:
# fitting ldc on fitness measure features
def fit_ldc(train_feature_matrix, train_labels):
    ldc = LinearDiscriminantAnalysis()
    ldc.fit(train_feature_matrix, train_labels)
    return ldc

In [0]:
# get prediction probability of neutral class
def get_neutral_prediction_probabilites(ldc, x, y):
    return np.asarray(ldc.predict_proba(x))[:, 0] 

In [0]:
# get the labels of entire dataset and neutral predictions per speaker
def get_neutral_subset_per_speaker(val_neutral_probs, train_neutral_probs, val_x, train_x):
    per_speaker_neutral_subset = {}
    for i in range(num_train_actors):
        per_speaker_neutral_subset[i] = []

    global_val_idx = 0
    global_train_idx = 0
    global_idx = 0
    changed_labels = [-1]*108

    for i in range(num_train_actors):
        if(train_neutral_probs[global_train_idx]>0.6):
            changed_labels[global_idx] = 0
            per_speaker_neutral_subset[i].append(train_x[global_train_idx])
        else:
            changed_labels[global_idx] = 1
        global_train_idx+=1
        global_idx+=1
        if(train_neutral_probs[global_train_idx]>0.6):
            changed_labels[global_idx] = 0
            per_speaker_neutral_subset[i].append(train_x[global_train_idx])
        else:
            changed_labels[global_idx] = 1
        global_train_idx+=1
        global_idx+=1
        if(val_neutral_probs[global_val_idx]>0.6):
            changed_labels[global_idx] = 0
            per_speaker_neutral_subset[i].append(val_x[global_val_idx])
        else:
            changed_labels[global_idx] = 1
        global_val_idx+=1
        global_idx+=1

    for i in range(num_train_actors):
        if(train_neutral_probs[global_train_idx]>0.6):
            changed_labels[global_idx] = 0
            per_speaker_neutral_subset[i].append(train_x[global_train_idx])
        else:
            changed_labels[global_idx] = 1
        global_train_idx+=1
        global_idx+=1
        if(train_neutral_probs[global_train_idx]>0.6):
            changed_labels[global_idx] = 0
            per_speaker_neutral_subset[i].append(train_x[global_train_idx])
        else:
            changed_labels[global_idx] = 1
        global_train_idx+=1
        global_idx+=1
        if(val_neutral_probs[global_val_idx]>0.6):
            changed_labels[global_idx] = 0
            per_speaker_neutral_subset[i].append(val_x[global_val_idx])
        else:
            changed_labels[global_idx] = 1
        global_val_idx+=1
        global_idx+=1

    return changed_labels, per_speaker_neutral_subset

In [0]:
# get normalization parameters per speaker
def get_speaker_normalizers(per_speaker_neutral_subset):
    per_speaker_normalizers = []

    for i in range(num_train_actors):
          if(len(per_speaker_neutral_subset[i])==0):
              per_speaker_normalizers.append(f0_ref)
          else:
              temp_arr = []
              for j in range(len(per_speaker_neutral_subset[i])):
                    temp_arr.append(np.mean(per_speaker_neutral_subset[i][j]))
              per_speaker_normalizers.append(np.mean(temp_arr))

    return per_speaker_normalizers  

In [0]:
# get percentage change in the number of labels changed in IFN iteration
def get_percentage_change_labels(changed_labels, prediction_labels):
    arr = list(np.asarray(changed_labels) - np.asarray(prediction_labels))
    num_changed_labels = len(prediction_labels)-arr.count(0)
    percentage_change = 100*(num_changed_labels/len(prediction_labels))
    # print(percentage_change) 
    return percentage_change

In [0]:
# normalize dataset per speaker according to per speaker normalizers
def normalize_datasets(sampled_emotional_subset, sampled_neutral_subset, per_speaker_normalizers):
    for actor_idx in range(len(sampled_neutral_subset)):
        sampled_neutral_subset[actor_idx] *= per_speaker_normalizers[actor_idx]
        # sampled_emotional_subset[actor_idx] *= f0_ref/per_speaker_normalizers[actor_idx]

    return sampled_emotional_subset, sampled_neutral_subset

In [59]:
# main script

prediction_labels = [-1]*108
sampled_emotional_subset, sampled_neutral_subset = get_sampled_data(training_emotional_subset, training_neutral_subset)
test_x, test_y = create_test_data(test_neutral_subset, test_emotional_subset)
ifn_iter = 0

while(True):
    print('------------ IFN Iteration '+str(ifn_iter)+' ---------------')
    trained_gmms = train_gmss(reference_neutral_features)    
    fitness_measures_emotional, fitness_measures_neutral = get_fitness_measures(sampled_emotional_subset, sampled_neutral_subset, trained_gmms)
    train_x, val_x, train_y, val_y = create_train_val_xy(fitness_measures_emotional, fitness_measures_neutral)
    ldc = fit_ldc(train_x, train_y)

    print('Validation accuracy: '+str(accuracy_score(val_y, ldc.predict(val_x))))
    print('Train accuracy: '+str(accuracy_score(train_y, ldc.predict(train_x))))
    print('Test accuracy: '+str(accuracy_score(test_y, ldc.predict(test_x))))

    val_neutral_probs = get_neutral_prediction_probabilites(ldc, val_x, val_y)
    train_neutral_probs = get_neutral_prediction_probabilites(ldc, train_x, train_y)

    changed_labels, per_speaker_neutral_subset = get_neutral_subset_per_speaker(val_neutral_probs, train_neutral_probs, val_x, train_x)
    per_speaker_normalizers = get_speaker_normalizers(per_speaker_neutral_subset)
    percentage_change = get_percentage_change_labels(changed_labels, prediction_labels)
    print('Percentage change: '+str(percentage_change))
    sampled_emotional_subset, sampled_neutral_subset = normalize_datasets(sampled_emotional_subset, sampled_neutral_subset, per_speaker_normalizers)

    if(ifn_iter>15):
          break

    ifn_iter+=1
    prediction_labels = changed_labels

------------ IFN Iteration 0 ---------------
Validation accuracy: 0.7222222222222222
Train accuracy: 0.6805555555555556
Test accuracy: 0.3898305084745763
Percentage change: 100.0
------------ IFN Iteration 1 ---------------
Validation accuracy: 0.8611111111111112
Train accuracy: 0.9583333333333334
Test accuracy: 0.9491525423728814
Percentage change: 44.44444444444444
------------ IFN Iteration 2 ---------------
Validation accuracy: 0.8888888888888888
Train accuracy: 0.9444444444444444
Test accuracy: 0.9491525423728814
Percentage change: 4.62962962962963
------------ IFN Iteration 3 ---------------
Validation accuracy: 1.0
Train accuracy: 0.9861111111111112
Test accuracy: 0.9491525423728814
Percentage change: 7.4074074074074066
------------ IFN Iteration 4 ---------------
Validation accuracy: 1.0
Train accuracy: 0.9861111111111112
Test accuracy: 0.9491525423728814
Percentage change: 0.0
------------ IFN Iteration 5 ---------------
Validation accuracy: 1.0
Train accuracy: 0.9861111111111